# Results Files Generation

In [1]:
import glob
import os
from tqdm import tqdm

## Generating Results based on Qrels

In [25]:
for year in ['2019']:
    dl_year = "dl" + year

    dir_path = f"TREC-DLs-files/TREC-DL-{year}"
    runs_path = dir_path + "/passages-runs/*"
    file_list = glob.glob(runs_path)

    # if need to remove 'input.'
    for file in file_list:
        try:
            os.rename(file, f"{runs_path.replace('*', '')}/{file.split('/')[3].split('.')[1]}")
        except:
            continue
    
    # to replace the old file paths with the new ones: without 'input'
    file_list = glob.glob(runs_path)

    # iterate over the files and generate the result files for DL and Resource qrels
    for file_path in file_list:
        run_id = file_path.split('/')[3]

        # TREC DL results
        # os.system(f'trec_eval -q {dir_path}/qrels-pass.txt {file_path} > TREC-DLs-results/{dl_year}/treceval/{run_id}.treceval')
        # os.system(f'trec_eval -m Rndcg -m ndcg_cut -c -q {dir_path}/qrels-pass.txt {file_path} > TREC-DLs-results/{dl_year}/ndcgeval/{run_id}.ndcgeval')

        # TREC FULL Synthetic (GPT-4) Results
        # os.system(f'trec_eval -q TestJudge-qrels/TestJudge_qrels_pass.txt {file_path} > TestJudge-results/{dl_year}/treceval/{run_id}.treceval')
        # os.system(f'trec_eval -m Rndcg -m ndcg_cut -c -q TestJudge-qrels/TestJudge_qrels_pass.txt {file_path} > TestJudge-results/{dl_year}/ndcgeval/{run_id}.ndcgeval')
        
        # TREC Partial Synthetic (GPT-4) (+ NIST from DL) Results
        os.system(f'trec_eval -q TestJudge-qrels/TestJudge_qrels_pass_withNIST.txt {file_path} > TestJudge-partial-results/{dl_year}/treceval/{run_id}.treceval')
        os.system(f'trec_eval -m Rndcg -m ndcg_cut -c -q TestJudge-qrels/TestJudge_qrels_pass_withNIST.txt {file_path} > TestJudge-partial-results/{dl_year}/ndcgeval/{run_id}.ndcgeval')


## Merging Results into One file

In [26]:
def get_file_names(dl_year, result_file_format):
    result_file_dl = f"final-results/all.pass.{dl_year}.{result_file_format}"
    result_file_partial = f"final-results/all.pass.{dl_year}.TestJudge.partial.{result_file_format}"
    result_file_full = f"final-results/all.pass.{dl_year}.TestJudge.full.{result_file_format}"
    return result_file_dl, result_file_partial, result_file_full

In [27]:
def get_file_paths(dl_year, result_file_format):
    results_path_dl = f"TREC-DLs-results/{dl_year}/{result_file_format}/*.{result_file_format}"
    results_path_partial = f"TestJudge-partial-results/{dl_year}/{result_file_format}/*.{result_file_format}"
    results_path_full = f"TestJudge-full-results/{dl_year}/{result_file_format}/*.{result_file_format}"
    return results_path_dl, results_path_partial, results_path_full

In [28]:
def write_merge_results(result_path, result_files):
    for result_file in tqdm(result_files):
        # print(result_file)
        run_id = result_file.split('/')[3].split('.')[0]
        result_file_data = open(result_file, 'r').readlines()
        for eachline in result_file_data:
            metric, qid, score = eachline.split('\t')
            metric, score = metric.strip(), score.strip()
            result_path.write(f"{run_id}\t{metric}\t{qid}\t{score}\n")

    result_path.close()

In [29]:
for year in ['2019', '2020', '2021', '2022', '2023']:
    for result_type in ['treceval', 'ndcgeval']:
        dl_year = "dl" + year
        result_file_dl, result_file_partial, result_file_full = get_file_names(dl_year, result_type)
        results_path_dl, results_path_partial, results_path_full = get_file_paths(dl_year, result_type)

        # Creating files for merged results
        result_all_dl = open(result_file_dl, 'w')
        result_all_partial = open(result_file_partial, 'w')
        result_all_full = open(result_file_full, 'w')

        # Loading files
        result_files_dl = glob.glob(results_path_dl)
        result_files_partial = glob.glob(results_path_partial)
        result_files_full = glob.glob(results_path_full)

        # merge and write dl result
        write_merge_results(result_all_dl, result_files_dl)
        # merge and write resource-partial result
        write_merge_results(result_all_partial, result_files_partial)
        # merge and write resource-full result
        write_merge_results(result_all_full, result_files_full)

100%|██████████| 35/35 [00:00<00:00, 328.45it/s]
